Divide aleatoriamente el conjunto de datos de cáncer de seno de Wisconsin3

en un subconjunto de
entrenamiento con el 60 % de los datos, un subconjunto de validación con el 20 % y un subconjunto
de prueba con el 20 % restante usando 0 como semilla para tu generador de números aleatorios.
Este conjunto de datos contiene 699 registros de tumores de seno, de los cuales 458 son benignos y
241 son malignos.

In [13]:
#Librerias
import pandas as pd

In [15]:

# URL del archivo de datos
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"

# Cargar los datos en un DataFrame
df = pd.read_csv(url, header=None)  # No tiene encabezados, por eso usamos header=None


In [20]:
# Definir nombres de columnas en español
column_names = [
    "ID_Muestra", "Grosor_Tumor", "Uniformidad_Tamaño_Celular", 
    "Uniformidad_Forma_Celular", "Adhesión_Marginal", "Tamaño_Célula_Epitelial", 
    "Núcleos_Desnudos", "Cromatina_Blanda", "Nucléolos_Normales", 
    "Mitosis_Celular", "Clase"
]

# Asignar los nombres de columna
df.columns = column_names

In [21]:
print(df.head(10))  # Mostrar las primeras 5 filas del DataFrame

   ID_Muestra  Grosor_Tumor  Uniformidad_Tamaño_Celular  \
0     1000025             5                           1   
1     1002945             5                           4   
2     1015425             3                           1   
3     1016277             6                           8   
4     1017023             4                           1   
5     1017122             8                          10   
6     1018099             1                           1   
7     1018561             2                           1   
8     1033078             2                           1   
9     1033078             4                           2   

   Uniformidad_Forma_Celular  Adhesión_Marginal  Tamaño_Célula_Epitelial  \
0                          1                  1                        2   
1                          4                  5                        7   
2                          1                  1                        2   
3                          8                  